In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

%load_ext google.colab.data_table

Authenticated


Aby przekonwertować ciąg na  obiekt daty , możesz użyć PARSE_DATE().  Jako pierwszy argument przyjmuje  ciąg formatujący , a jako drugi ciąg przypominający datę.

https://cloud.google.com/bigquery/docs/reference/standard-sql/date_functions#supported_format_elements_for_date

Aby zrobić coś przeciwnego, tj. przekonwertować datę  na ciąg znaków, możesz użyć FORMAT_DATE(). Jako pierwszy argument przyjmuje ciąg formatu, a jako drugi argument wartość daty.

Aby uzyskać bieżącą datę (dzisiaj) jako wartość daty, użyj CURRENT_DATE().

Aby wyodrębnić część daty (np. rok, miesiąc, dzień) z obiektu daty, użyj EXTRACT().

AppsFlyer

In [ ]:
%%bigquery --project produkcja-mobile

SELECT 
event_time, -- 2021-07-03 23:44:22.448 / UTC
cast(format_date('%Y%m%d', extract(date from cast(event_time as DATETIME))) as string) as event_time, -- 20210703 / UTC

event_time_selected_timezone, -- 2021-07-04 01:44:22.448+0200
CAST(DATE(PARSE_TIMESTAMP('%F %H:%M:%E3S%z', event_time_selected_timezone), "Europe/Warsaw") as DATE) event_time_selected_timezone, -- 2021-07-04 / DATE
CAST(DATE(PARSE_TIMESTAMP('%F %H:%M:%E3S%z', event_time_selected_timezone), "Europe/Warsaw") as STRING) event_time_selected_timezone, -- 2021-07-04 / STRING
CAST(FORMAT_TIMESTAMP("%Y-%m-%dT%X", DATETIME(TIMESTAMP_TRUNC(PARSE_TIMESTAMP('%F %H:%M:%E3S%z', event_time_selected_timezone), second), "Europe/Warsaw")) as DATETIME) event_time_selected_timezone, -- 2021-07-04T01:44:22
CAST(FORMAT_TIMESTAMP("%Y-%m-%dT%X", DATETIME(TIMESTAMP_TRUNC(PARSE_TIMESTAMP('%F %H:%M:%E3S%z', event_time_selected_timezone), second), "Europe/Warsaw")) as STRING) event_time_selected_timezone, -- 2021-07-04T01:44:22

install_time_selected_timezone,
CAST(DATE(PARSE_TIMESTAMP('%F %H:%M:%E3S%z', install_time_selected_timezone), "Europe/Warsaw") as DATE) install_time_selected_timezone, -- 2021-07-04 / DATE
CAST(DATE(PARSE_TIMESTAMP('%F %H:%M:%E3S%z', install_time_selected_timezone), "Europe/Warsaw") as STRING) install_time_selected_timezone, -- 2021-07-04 / STRING
CAST(FORMAT_TIMESTAMP("%Y-%m-%dT%X", DATETIME(TIMESTAMP_TRUNC(PARSE_TIMESTAMP('%F %H:%M:%E3S%z', install_time_selected_timezone), second), "Europe/Warsaw")) as DATETIME) install_time_selected_timezone, -- 2021-07-04T01:44:22
CAST(FORMAT_TIMESTAMP("%Y-%m-%dT%X", DATETIME(TIMESTAMP_TRUNC(PARSE_TIMESTAMP('%F %H:%M:%E3S%z', install_time_selected_timezone), second), "Europe/Warsaw")) as STRING) install_time_selected_timezone, -- 2021-07-04T01:44:22


FROM `eobuwie-181013.01_AppsFlyer_EOB.mobile_devices_events` 
where partition_time = date_add(CURRENT_DATE(), interval -1 DAY)
LIMIT 4


,event_time,event_time_1,event_time_selected_timezone,event_time_selected_timezone_1,event_time_selected_timezone_2,event_time_selected_timezone_3,event_time_selected_timezone_4,install_time_selected_timezone,install_time_selected_timezone_1,install_time_selected_timezone_2,install_time_selected_timezone_3,install_time_selected_timezone_4
0,2021-09-17 23:27:13.682,20210917,2021-09-18 01:27:13.682+0200,2021-09-18,2021-09-18,2021-09-18 01:27:13,2021-09-18T01:27:13,2021-09-18 01:11:37.000+0200,2021-09-18,2021-09-18,2021-09-18 01:11:37,2021-09-18T01:11:37
1,2021-09-17 23:28:44.351,20210917,2021-09-18 01:28:44.351+0200,2021-09-18,2021-09-18,2021-09-18 01:28:44,2021-09-18T01:28:44,2021-07-21 17:52:14.384+0200,2021-07-21,2021-07-21,2021-07-21 17:52:14,2021-07-21T17:52:14
2,2021-09-17 23:29:46.023,20210917,2021-09-18 01:29:46.023+0200,2021-09-18,2021-09-18,2021-09-18 01:29:46,2021-09-18T01:29:46,2021-05-31 19:52:18.305+0200,2021-05-31,2021-05-31,2021-05-31 19:52:18,2021-05-31T19:52:18
3,2021-09-17 23:30:08.001,20210917,2021-09-18 01:30:08.001+0200,2021-09-18,2021-09-18,2021-09-18 01:30:08,2021-09-18T01:30:08,2021-05-18 21:08:41.973+0200,2021-05-18,2021-05-18,2021-05-18 21:08:41,2021-05-18T21:08:41


Firebase

In [ ]:
%%bigquery --project produkcja-mobile

select
    event_date as date, # 20210601
    format_date('%Y',parse_date("%Y%m%d",event_date)) as year, # 2021
    format_date('%G',parse_date("%Y%m%d",event_date)) as iso_year, #2021
    format_date('%Y%m',parse_date("%Y%m%d",event_date)) as month_of_year, # 202106
    format_date('%m',parse_date("%Y%m%d",event_date)) as month_of_the_year, #06
    format_date('%Y%U',parse_date("%Y%m%d",event_date)) as week_of_year, #202122
    format_date('%U',parse_date("%Y%m%d",event_date)) as week_of_the_year, #22
    format_date('%W',parse_date("%Y%m%d",event_date)) as iso_week_of_the_year, #22
    format_date('%G%W',parse_date("%Y%m%d",event_date)) as iso_week_of_iso_year, #202122
    format_date('%d',parse_date("%Y%m%d",event_date)) as day_of_the_month, #01
    format_date('%w',parse_date("%Y%m%d",event_date)) as day_of_week, #2
    format_date('%A',parse_date("%Y%m%d",event_date)) as day_of_week_name, #Tuesday
    concat(event_date,cast(format("%02d",extract(hour from timestamp_micros(event_timestamp))) as string)) as hour_of_day, #2021060105
    format("%02d",extract(hour from timestamp_micros(event_timestamp))) as hour, #05
    format("%02d",extract(minute from timestamp_micros(event_timestamp))) as minute, #35
    concat(concat(event_date,cast(format("%02d",extract(hour from timestamp_micros(event_timestamp))) as string)),format("%02d",extract(minute from timestamp_micros(event_timestamp)))) as date_hour_and_minute # 202106010535
from
    `simoahava-com.analytics_206575074.events_*`
where
    _table_suffix between '20210627' and format_date('%Y%m%d',date_sub(current_date(), interval 1 day))
LIMIT 5

,date,year,iso_year,month_of_year,month_of_the_year,week_of_year,week_of_the_year,iso_week_of_the_year,iso_week_of_iso_year,day_of_the_month,day_of_week,day_of_week_name,hour_of_day,hour,minute,date_hour_and_minute
0,20210903,2021,2021,202109,09,202135,35,35,202135,03,5,Friday,2021090306,06,41,202109030641
1,20210903,2021,2021,202109,09,202135,35,35,202135,03,5,Friday,2021090306,06,41,202109030641
2,20210903,2021,2021,202109,09,202135,35,35,202135,03,5,Friday,2021090300,00,35,202109030035
3,20210903,2021,2021,202109,09,202135,35,35,202135,03,5,Friday,2021090300,00,35,202109030035
4,20210903,2021,2021,202109,09,202135,35,35,202135,03,5,Friday,2021090300,00,35,202109030035


Kod krajów & Strefy czasowe:

https://en.wikipedia.org/wiki/List_of_tz_database_time_zones

https://www.pascallandau.com/bigquery-snippets/convert-timestamp-date-datetime-to-different-timezone/

In [ ]:
%%bigquery --project produkcja-mobile

WITH examples AS (
  SELECT TIMESTAMP("2018-04-08T15:50:10+00:00") AS timestamp # Daylight saving time
  UNION ALL SELECT TIMESTAMP("2018-03-08T15:50:10+00:00")    # Standard time
)
SELECT 
  timestamp, 
  DATETIME(timestamp) as datetime,
  DATE(timestamp) as date,
  TIME(timestamp) as time,
  DATETIME(timestamp, "Europe/Berlin") as datetime_berlin,
  DATE(timestamp, "Europe/Berlin") as date_berlin,
  TIME(timestamp, "Europe/Berlin") as time_berlin
FROM examples

,timestamp,datetime,date,time,datetime_berlin,date_berlin,time_berlin
0,2018-04-08 15:50:10+00:00,2018-04-08 15:50:10,2018-04-08,15:50:10,2018-04-08 17:50:10,2018-04-08,17:50:10
1,2018-03-08 15:50:10+00:00,2018-03-08 15:50:10,2018-03-08,15:50:10,2018-03-08 16:50:10,2018-03-08,16:50:10


Funkcje na datach: https://cloud.google.com/bigquery/docs/reference/standard-sql/functions-and-operators#date

Funkcje na Time: https://cloud.google.com/bigquery/docs/reference/standard-sql/functions-and-operators#time

Funkcje na Datetime: https://cloud.google.com/bigquery/docs/reference/standard-sql/functions-and-operators#datetime

In [ ]:
# Wczorajszy dzień
where
_TABLE_SUFFIX BETWEEN 
FORMAT_DATE('%Y%m%d',DATE_SUB(CURRENT_DATE(), INTERVAL 1 DAY)) and
FORMAT_DATE('%Y%m%d',DATE_SUB(CURRENT_DATE(), INTERVAL 1 DAY))

# Odwołanie do tabeli Intraday oraz wskazanie na zakres dat 14 dni + Intraday
where regexp_extract(_table_suffix, r'[0-9]+') between format_date('%Y%m%d',date_sub(current_date(), interval 14 day))
and format_date('Y%m%d%', current_date())

# Wskazanie na zakres dat 14 dni 
where _table_suffix between format_date('%Y%m%d',date_sub(current_date(), interval 14 day))

# Od 20210501 do tydzień temu
_TABLE_SUFFIX BETWEEN '20210501' and
FORMAT_DATE('%Y%m%d',DATE_SUB(CURRENT_DATE(), INTERVAL 7 DAY))



https://www.ga4bigquery.com/tutorial-how-to-query-multiple-analytics-events-tables-with-table-suffix-ga4/

Statyczny zakres dat:

In [ ]:
%%bigquery --project produkcja-mobile

select
    *
from
    `simoahava-com.analytics_206575074.events_*`
where
    _table_suffix between '20210103' and '20210203'

Dynamiczny zakres dat (ostatnie 30 dni):

In [ ]:
%%bigquery --project produkcja-mobile

select
    *
from
    `simoahava-com.analytics_206575074.events_*`
where
    _table_suffix between format_date('%Y%m%d',date_sub(current_date(), interval 30 day))
    and format_date('%Y%m%d',date_sub(current_date(), interval 1 day))

Daty statyczne i dynamiczne (poprzedni dzień):

In [ ]:
%%bigquery --project produkcja-mobile

select
    *
from
    `simoahava-com.analytics_206575074.events_*`
where
    _table_suffix between '20210103' 
    and format_date('%Y%m%d',date_sub(current_date(), interval 1 day))

In [ ]:
%%bigquery --project produkcja-mobile

select
    event_date as date,
    count(event_name) as sessions
from
    `simoahava-com.analytics_206575074.events_*`
where
    # ostatnie 3 dni
    regexp_extract(_table_suffix, r'[0-9]+') between format_date('%Y%m%d',date_sub(current_date(), interval 3 day)) and format_date('%Y%m%d', current_date())

    #regexp_extract(_table_suffix, '[0-9]+') between '20210501' and format_date('%Y%m%d', current_date()) and event_name = 'session_start'
group by 
    date
order by
    date desc
LIMIT 5

,date,sessions
0,20210918,6537
1,20210917,57963
2,20210916,68312
3,20210915,69426


Dane za wczoraj:

In [ ]:
%%bigquery --project produkcja-mobile

select
    *
from
    `simoahava-com.analytics_206575074.events_*`

where _TABLE_SUFFIX BETWEEN
FORMAT_DATE('%Y%m%d',DATE_SUB(CURRENT_DATE(), INTERVAL 1 DAY)) and
FORMAT_DATE('%Y%m%d',DATE_SUB(CURRENT_DATE(), INTERVAL 1 DAY))

https://cloud.google.com/bigquery/docs/reference/standard-sql/date_functions#supported_format_elements_for_date

AppsFlyer

In [ ]:
%%bigquery --project produkcja-mobile

SELECT
attributed_touch_time,
attributed_touch_time_selected_timezone,
attributed_touch_type,
contributor_1_touch_time,
contributor_2_touch_time,
contributor_3_touch_time,

custom_data,
download_time,
download_time_selected_timezone,
event_time,
event_time_selected_timezone,
install_time,
install_time_selected_timezone,
selected_timezone

FROM `eobuwie-181013.01_AppsFlyer_EOB.mobile_devices_events` 
where extract(date from cast(event_time as DATETIME)) = date_add(CURRENT_DATE(), interval -1 DAY)
order by 1 desc
LIMIT 5

,attributed_touch_time,attributed_touch_time_selected_timezone,attributed_touch_type,contributor_1_touch_time,contributor_2_touch_time,contributor_3_touch_time,custom_data,download_time,download_time_selected_timezone,event_time,event_time_selected_timezone,install_time,install_time_selected_timezone,selected_timezone
0,2021-09-13 23:57:32.139,2021-09-14 01:57:32.139+0200,click,2021-09-13 10:38:28.858,None,None,None,2021-09-13 23:57:38.000,2021-09-14 01:57:38.000+0200,2021-09-13 23:58:21.864,2021-09-14 01:58:21.864+0200,2021-09-13 23:58:21.864,2021-09-14 01:58:21.864+0200,Europe/Warsaw
1,2021-09-13 23:56:27.851,2021-09-14 01:56:27.851+0200,click,None,None,None,None,2021-09-02 18:04:10.000,2021-09-02 20:04:10.000+0200,2021-09-13 23:57:28.771,2021-09-14 01:57:28.771+0200,2021-09-13 23:57:28.771,2021-09-14 01:57:28.771+0200,Europe/Warsaw
2,2021-09-13 23:51:09.310,2021-09-14 01:51:09.310+0200,click,None,None,None,None,2021-09-13 23:52:24.000,2021-09-14 01:52:24.000+0200,2021-09-13 23:55:22.025,2021-09-14 01:55:22.025+0200,2021-09-13 23:54:12.690,2021-09-14 01:54:12.690+0200,Europe/Warsaw
3,2021-09-13 23:51:09.310,2021-09-14 01:51:09.310+0200,click,2021-09-10 14:30:27.556,None,None,None,2021-09-13 23:52:24.000,2021-09-14 01:52:24.000+0200,2021-09-13 23:54:12.690,2021-09-14 01:54:12.690+0200,2021-09-13 23:54:12.690,2021-09-14 01:54:12.690+0200,Europe/Warsaw
4,2021-09-13 23:48:31.788,2021-09-14 01:48:31.788+0200,click,2021-09-13 05:51:52.616,None,None,None,2021-09-13 05:53:59.000,2021-09-13 07:53:59.000+0200,2021-09-13 23:48:53.692,2021-09-14 01:48:53.692+0200,2021-09-13 23:48:53.692,2021-09-14 01:48:53.692+0200,Europe/Warsaw


In [ ]:
%%bigquery --project produkcja-mobile

SELECT
cast(format_date('%Y%m%d', extract(date from cast(event_time as DATETIME))) as string) event_date,
extract(date from cast(event_time as DATETIME)) true_date,
format_date('%Y%m',extract(date from cast(event_time as DATETIME))) month_of_year,
format_date('%Y',extract(date from cast(event_time as DATETIME))) year,
format_date('%U',extract(date from cast(event_time as DATETIME))) week_of_the_year
FROM `eobuwie-181013.01_AppsFlyer_EOB.mobile_devices_events` 
-- where extract(date from cast(event_time as DATETIME)) between '2021-06-15' and '2021-09-13' Pełny zakres dat
where extract(date from cast(event_time as DATETIME)) = date_add(CURRENT_DATE(), interval -1 DAY) # Wczorajszy dzień 

group by 1,2,3,4,5
order by 1 desc
LIMIT 5

,event_date,true_date,month_of_year,year,week_of_the_year
0,20210913,2021-09-13,202109,2021,37
